## 49단계: Dataset 클래스와 전처리

> 이전 단계처럼 데이터셋을 처리할 경우, 대규모 데이터셋을 처리할 때는 문제가 됩니다. 예를 들어 데이터가 100만 개라면 메모리가 부족할 수 있습니다. 이러한 문제에 대응할 수 있도록 Dataset 클래스를 만들려 합니다.

### 49.1 Dataset 클래스 구현

In [1]:
# dezero/datasets.py

import numpy as np

class Dataset:
    def __init__(self, train=True):
        self.train = train
        self.data = None
        self.labels = None
        self.prepare()
        
    def __getitem__(self, index):
        assert np.isscalar(index), "Index must be a scalar"
        if self.label is None:
            return self.data[index], None
        else:
            return self.data[index], self.labels[index]
        
    def __len__(self):
        return len(self.data)
    
    def prepare(self):
        pass

Dataset을 상속하여 스파이럴 데이터셋을 구현해보자.

In [2]:
# dezero/datasets.py

def get_spiral(train=True):
    # Implementation of the spiral dataset generation
    ...

class Spiral(Dataset):
    def prepare(self):
        self.data, self.label = get_spiral(self.train)

In [3]:
import dezero

train_set = dezero.datasets.Spiral(train=True)
print(train_set[0])
print(len(train_set))

(array([-0.13981389, -0.00721657], dtype=float32), np.int64(1))
300


### 49.2 큰 데이터셋의 경우

아래 클래스와 같이 `__getitem__`이 호출되는 시점에 데이터를 읽을 수 있다.

In [4]:
class BigData(Dataset):
    def __getitem__(self, index):
        x = np.load('data/{}.npy'.format(index))
        t = np.load('label/{}.npy'.format(index))
        return x, t
    
    def __len__(self):
        return 1_000_000

### 49.3 데이터 이어 붙이기

In [5]:
train_set = dezero.datasets.Spiral()

batch_index = [0, 1, 2]  # 0에서 2번째까지의 데이터를 꺼냄
batch = [train_set[i] for i in batch_index]
# batch = [(data_0, label_0), (data_1, label_1), (data_2, label_2)]

x = np.array([example[0] for example in batch])
t = np.array([example[1] for example in batch])

print(x.shape)
print(t.shape)

(3, 2)
(3,)


### 49.4 학습 코드

In [6]:
# 48단계의 학습 코드 수정

import math
import numpy as np
import dezero
from dezero import optimizers
import dezero.functions as F
from dezero.models import MLP

max_epoch = 300
batch_size = 30
hidden_size = 10
lr = 1.0

train_set = dezero.datasets.Spiral()  # Spiral 데이터셋을 불러옴
model = MLP((hidden_size, 3))
optimizer = optimizers.SGD(lr).setup(model)

data_size = len(x)
max_iter = math.ceil(data_size / batch_size)

for epoch in range(max_epoch):
    index = np.random.permutation(data_size)
    sum_loss = 0
    
    for i in range(max_iter):
        # 미니배치 꺼내기
        batch_index = index[i * batch_size:(i + 1) * batch_size]
        batch = [train_set[j] for j in batch_index]
        batch_x = np.array([example[0] for example in batch])  # 이어붙이기
        batch_t = np.array([example[1] for example in batch])
        
        y = model(batch_x)
        loss = F.softmax_cross_entropy(y, batch_t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        
        sum_loss += float(loss.data) * len(batch_x)
        
    avg_loss = sum_loss / data_size
    print(f'epoch {epoch + 1}, loss {avg_loss:.2f}')

epoch 1, loss 1.08
epoch 2, loss 0.76
epoch 3, loss 0.71
epoch 4, loss 0.69
epoch 5, loss 0.67
epoch 6, loss 0.66
epoch 7, loss 0.65
epoch 8, loss 0.64
epoch 9, loss 0.63
epoch 10, loss 0.63
epoch 11, loss 0.62
epoch 12, loss 0.61
epoch 13, loss 0.61
epoch 14, loss 0.60
epoch 15, loss 0.59
epoch 16, loss 0.58
epoch 17, loss 0.58
epoch 18, loss 0.57
epoch 19, loss 0.56
epoch 20, loss 0.56
epoch 21, loss 0.55
epoch 22, loss 0.54
epoch 23, loss 0.53
epoch 24, loss 0.53
epoch 25, loss 0.52
epoch 26, loss 0.51
epoch 27, loss 0.50
epoch 28, loss 0.49
epoch 29, loss 0.48
epoch 30, loss 0.48
epoch 31, loss 0.47
epoch 32, loss 0.46
epoch 33, loss 0.45
epoch 34, loss 0.44
epoch 35, loss 0.43
epoch 36, loss 0.42
epoch 37, loss 0.41
epoch 38, loss 0.40
epoch 39, loss 0.40
epoch 40, loss 0.39
epoch 41, loss 0.38
epoch 42, loss 0.37
epoch 43, loss 0.36
epoch 44, loss 0.35
epoch 45, loss 0.34
epoch 46, loss 0.34
epoch 47, loss 0.33
epoch 48, loss 0.32
epoch 49, loss 0.31
epoch 50, loss 0.31
epoch 51,

### 49.5 데이터셋 전처리

머신러닝에서는 모델에 데이터를 입력하기 전에 데이터를 가공할 때가 많다. 이에 대응하기 위해 Dataset에 전처리 기능을 추가한다.

In [7]:
# dezero/datasets.py

class Dataset:
    def __init__(self, train=True, transform=None, target_transform=None):
        self.train = train
        self.transform = transform
        self.target_transform = target_transform
        if self.transform is None:
            self.transform = lambda x: x  # 만약 전처리 함수가 None이면 받은 값을 그대로 반환
        if self.target_transform is None:
            self.target_transform = lambda x: x

        self.data = None
        self.label = None
        self.prepare()

    def __getitem__(self, index):
        assert np.isscalar(index)
        if self.label is None:
            return self.transform(self.data[index]), None
        else:
            return self.transform(self.data[index]),\
                   self.target_transform(self.label[index])

    def __len__(self):
        return len(self.data)

    def prepare(self):
        pass

In [8]:
# 예시 1

def f(x):
    y = x / 2.0
    return y

train_set = dezero.datasets.Spiral(transform=f)

In [9]:
# 예시 2: dezero의 transforms 모듈엔 다양한 변환이 정의되어 있다

from dezero import transforms

f = transforms.Normalize(mean=0.0, std=2.0)
train_set = dezero.datasets.Spiral(transform=f)

In [10]:
# 예시 3: 여러 변환을 합쳐 사용할 수도 있다

f = transforms.Compose([transforms.Normalize(mean=0.0, std=2.0),
                        transforms.AsType(np.float64)])